In [1]:
import pandas
import json
from pandas.io.json import json_normalize
from datetime import datetime
pandas.set_option('display.max_columns',None)
pandas.set_option('display.max_rows',None)
import requests
import itertools
from bs4 import BeautifulSoup
import numpy

# Scraper for Balance Sheet

In [113]:
ticker_list = ['TSLA','MSFT','SQ','DNMR']
liab_df = pandas.DataFrame()
asset_df = pandas.DataFrame()

for symbol in ticker_list:
    url = "https://www.marketwatch.com/investing/stock/"+symbol+"/financials/balance-sheet"
    marketwatch = BeautifulSoup(requests.get(url).text,"lxml")
    try:
        liabilities = pandas.read_html(str(marketwatch.find_all("table")[5]))
        liabilities = liabilities[0].dropna(axis=0, thresh=4)
        liabilities['ticker'] = symbol
        liab_df = pandas.concat([liab_df,liabilities],axis=0,sort=True)
    except Exception:
        pass
    try:
        assets = pandas.read_html(str(marketwatch.find_all("table")[4]))
        assets = assets[0].dropna(axis=0, thresh=4)
        assets['ticker'] = symbol
        asset_df = pandas.concat([asset_df,assets],axis=0,sort=True)
    except Exception:
        pass
    
balance_sheets = pandas.concat([liab_df,asset_df],sort=True)
balance_sheets = pandas.melt(balance_sheets, id_vars=['Item Item','ticker'])
balance_sheets.columns = ['LineItem','Ticker','Period','Value']
balance_sheets = balance_sheets.dropna()
balance_sheets['Length'] = balance_sheets['Value'].str.len()
balance_sheets = balance_sheets.assign(Multiplier=balance_sheets.apply(lambda x: x['Value'][x['Length']-1:], axis=1))
balance_sheets['Neg?'] = numpy.where(balance_sheets['Multiplier'] == ')','yes','no')
balance_sheets['key'] = balance_sheets.Value.str.replace(r"[0-9/./()]",'')
balance_sheets['Stripped_Value'] = balance_sheets.Value.str.replace(r"[^\d.]",'')
balance_sheets['Stripped_Value'] = pandas.to_numeric(balance_sheets['Stripped_Value'])
balance_sheets['Real_Value'] = numpy.where(balance_sheets.key=='B',balance_sheets['Stripped_Value']*1000000000,
                                           numpy.where(balance_sheets.key=='M',balance_sheets['Stripped_Value']*1000000,
                                                 numpy.where(balance_sheets.key=='K',balance_sheets['Stripped_Value']*1000,
                                                       numpy.where(balance_sheets.key=='%',balance_sheets['Stripped_Value']/100,
                                                             numpy.where(balance_sheets.key=='-%',balance_sheets['Stripped_Value']/-100,
                                                                   numpy.where(balance_sheets.key=='',balance_sheets['Stripped_Value'],
                                                                               numpy.where(balance_sheets.key=='-',balance_sheets['Stripped_Value'],0)))))))

balance_sheets = balance_sheets[(['LineItem','Ticker','Period','Real_Value'])]
balance_sheets.pivot_table(index=(['Period','Ticker']), columns='LineItem', values='Real_Value').reset_index()
# .to_csv('balance_sheets.csv')

# IexFinance API construct below
### Have not perfected yet, use webscraper above

In [ ]:

# # url = 'https://cloud.iexapis.com/stable/'
# url = 'https://sandbox.iexapis.com/stable/'
# symbol = 'TSLA'
# ticker_list = ['TSLA','MSFT']

# with open('iex_config.json') as config_file:
#     config_data = json.load(config_file)

# # iextoken = config_data['token']
# iextoken = config_data['sandbox']
# stats_df = pandas.DataFrame()
# for ticker in ticker_list:
#     stats_url = url + 'stock/' + ticker + '/stats?token=' + iextoken
#     stats_json = requests.get(stats_url).json()
#     stats_norm = json_normalize(stats_json)
#     stats_df = pandas.concat([stats_df,stats_norm],axis=0)
# stats_df.head(10)
# ## only useful for real time bid/ask spreads

# book_df = pandas.DataFrame()
# for ticker in ticker_list:
#     book_url = url + 'stock/' + ticker + '/book?token=' + iextoken
#     book_json = requests.get(book_url).json()
#     book_norm = json_normalize(book_json)
#     book_df = pandas.concat([book_df,book_norm],axis=0)
# book_df.head(10)
# company_df = pandas.DataFrame()
# for ticker in ticker_list:
#     company_url = url + 'stock/' + ticker + '/company?token=' + iextoken
#     company_json = requests.get(company_url).json()
#     company_norm = json_normalize(company_json)
#     company_df = pandas.concat([company_df,company_norm],axis=0)
# company_df.head(10)
# prices_df = pandas.DataFrame()
# for ticker in ticker_list:
#     prices_url = url + 'stock/' + ticker + '/chart/2y?chartCloseOnly=True&token=' + iextoken
#     prices_json = requests.get(prices_url).json()
#     prices_norm = json_normalize(prices_json)
#     prices_norm['symbol'] = ticker
#     prices_df = pandas.concat([prices_df,prices_norm],axis=0)
# prices_df.head(10)
# peers_df = pandas.DataFrame()
# for ticker in ticker_list:
#     peers_url = url + 'stock/' + ticker + '/peers?token=' + iextoken
#     peers_json = requests.get(peers_url).json()
#     peers_norm = pandas.DataFrame(data={'symbol': ticker, 'comps': [peers_json]})
#     peers_df = pandas.concat([peers_df,peers_norm],axis=0)
# peers_df.head(10)
# bal_df = pandas.DataFrame()
# for ticker in ticker_list:
#     bal_url = url + 'stock/' + ticker + '/balance-sheet?period=quarter?last=4&token=' + iextoken
#     bal_json = requests.get(bal_url).json()
#     bal_norm = json_normalize(bal_json,'balancesheet')
#     bal_df = pandas.concat([bal_df,bal_norm],axis=0)
# bal_df.head(10)
# inc_df = pandas.DataFrame()
# for ticker in ticker_list:
#     inc_url = url + 'stock/' + ticker + '/income?period=annual?last=4&token=' + iextoken
#     inc_json = requests.get(inc_url).json()
#     inc_norm = json_normalize(inc_json,'income')
#     inc_df = pandas.concat([inc_df,inc_norm],axis=0)
# inc_df.head(10)
# ind_df = pandas.DataFrame()
# tag = 'Airlines'
# for ticker in ticker_list:
#     ind_url = url + 'stock/market/collection/tag?collectionName='+ tag +'&token=' + iextoken
#     ind_json = requests.get(ind_url).json()
#     ind_norm = json_normalize(ind_json)
#     ind_df = pandas.concat([ind_df,ind_norm],axis=0)
# ind_df.head(10)

# list(itertools.chain.from_iterable(peers_df.comps))
# combined_df = pandas.concat([pandas.concat([pandas.concat([pandas.concat([bal_df,inc_df],axis=1),company_df],axis=1),
#                               stats_df],axis=1),peers_df],axis=1)